# Import libraries

In [1]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import concurrent.futures
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv

In [2]:
main_start = time.time()

# Store Information

In [3]:
get_urls_list = []
raw_urls_list = []
clean_urls_list = []
raw_element_dictionary = dict()
df_element_list = list()

# Open driver

In [4]:
session = requests.Session()
driver = webdriver.Chrome()

# Save CSV Func

In [5]:
def save_to_csv(info_dict_list,file_name):
    """a function that prints a csv starting from a dictionarires list 
        main(list_dict) """
    # we need at least 2 dict to create a csv file, so I made a df from 2 dict (the first 2 dict that I have)
    df = pd.DataFrame([info_dict_list[0],info_dict_list[1]])
    df.to_csv(file_name, index=False)
    #
    for i in  range(2, len(info_dict_list)): #
        df = pd.read_csv(file_name)
        #I don't know if we are already cheacking for duplicates, but this will do that
        if info_dict_list[i]["id"] not in df["id"].array:
            #open csv in append mode ("a")
            file = open(file_name, "a")
            #use csv library to add a row to the csv file. The row has to be in a list format, so I've used the .values() method of the ditionary class
            csv.writer(file).writerow(info_dict_list[i].values())           
    #at the end close the csv file
    file.close

# Get Urls_scrape_list

In [6]:
property_type = ['house', 'apartment']
for x in range(1,4):
    for type in property_type:
        urls = f'https://www.immoweb.be/en/search/{type}/for-sale?countries=BE&page={x}&orderBy=relevance'
        get_urls_list.append(urls)

# Get elements Urls list

In [7]:
def get_elements_urls_list(url):
    #Open the page
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    
    #Get the soup
    s_links = soup.select('li.search-results__item a')
    #Get the links
    
    for element in s_links:
        link = element["href"]
        #clean the links
        if '/agency' in link or '/en/classified/' not in link: 
            pass

        elif 'https://www.immoweb.be' not in link:
            link = 'https://www.immoweb.be' + link
            if link not in clean_urls_list:
                clean_urls_list.append(link)
        else:
            if link not in clean_urls_list:
                clean_urls_list.append(link)
    return

In [8]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(get_elements_urls_list,get_urls_list)

In [9]:
print(len(clean_urls_list))
print(clean_urls_list)

61
['https://www.immoweb.be/en/classified/house/for-sale/evere/1140/10134094?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/8556707', 'https://www.immoweb.be/en/classified/house/for-sale/roosdaal/1760/10143558?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/apartment-block/for-sale/forest/1190/10142968?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/house/for-sale/zottegem/9620/10126151?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/herenthout/2270/10127053?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/villa/for-sale/rixensart/1330/10127823?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/house/for-sale/grez-doiceau/1390/10127813?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/house/for-sale/ixelles/1050/10088204?searchId=633ad23ea8643', 'https://www.immoweb.be/en/classified/house/for-sale/tervuren/3080/10126511?searchId=633ad23ea8643', 'ht

In [10]:
#save_to_csv(clean_urls_list,'urls_list.csv')

# Get Elements_properties_raw

In [11]:
def get_property(url):
    req_element = session.get(url)
    soup = BeautifulSoup(req_element.text, 'html.parser')
    element_info = soup.select('div.classified script')
    text = element_info[0].text
    return json.loads("".join(("".join(text.split('=')[1:]).split(";")[:-1])))

In [12]:
start = time.time()
for index,url in enumerate(clean_urls_list):
    #start = time.time()
    raw_element_dictionary[url] = get_property(url)
    #end = time.time()
    #print('scraped',index,' time ',end-start,url)
end = time.time()
print(end-start)

20.697983741760254


runs under 12 sec

In [13]:
start = time.time()  #Test
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(get_property, clean_urls_list)
end = time.time()  #Test
print(end-start)  #Test

6.171714782714844


runs under 4 sec

# Save into json

In [14]:
with open("raw_element_dictionary.json","w") as raw_element_dictionary_file:
    raw_element_dictionary_file.write(json.dumps(raw_element_dictionary))

# Get Elements_properties_list

In [15]:
def get_elements_properties(element):
    element_data = {
    'id': element['id'],
    'url': list(raw_element_dictionary.keys())[index],
    'Locality': element['property']['location']['postalCode'] if element['property']['location'] is not None else None,
    'Type of property': element['property']['type'] if element['property']['type'] is not None else None,
    'Subtype of property': element['property']['subtype'] if element['property']['subtype'] is not None else None,
    'Price': element['price']['mainValue'] if element['price'] is not None else None,
    'Type of sale': element['price']['type'] if element['price'] is not None else None,
    'Number of rooms': element['property']['roomCount'] if element['property']  is not None else None,
    'Area': element['property']['netHabitableSurface'] if element['property'] is not None else None,
    'Fully equipped kitchen': element['property']['kitchen']['type'] if element['property']['kitchen'] is not None else None,
    'Furnished': element['transaction']['sale']['isFurnished'] if element['transaction']['sale'] is not None else None,
    'Open fire': element['property']['fireplaceExists'] if element['property'] is not None else None,
    'Terrace': element['property']['hasTerrace'] if element['property'] is not None else None,
    'Terrace area': element['property']['terraceSurface'] if element['property'] is not None else None,
    'Garden': element['property']['hasGarden'] if element['property'] is not None else None,
    'Garden area': element['property']['gardenSurface'] if element['property'] is not None else None,
    'Surface of the land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Surface area of the plot of land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Number of facades ': element['property']['building']['facadeCount'] if element['property']['building'] is not None else 1,
    'Swimming pool': element['property']['hasSwimmingPool'] if element['property'] is not None else None,
    'State of the building': element['property']['building']['condition'] if element['property']['building'] is not None else None
    }
    return element_data

In [16]:
start = time.time()  #Test

end = time.time()  #Test
print(end-start)  #Test

1.811981201171875e-05


In [17]:
start = time.time()  #Test
for index,element in enumerate(raw_element_dictionary.values()):
    df_element_list.append(get_elements_properties(element))
    #print(index,'Done')  #,list(main_dictionary.keys())[index])
end = time.time()  #Test
print(end-start)  #Test

0.007098197937011719


In [18]:
print (df_element_list)

[{'id': 10134094, 'url': 'https://www.immoweb.be/en/classified/house/for-sale/evere/1140/10134094?searchId=633ad23ea8643', 'Locality': '1140', 'Type of property': 'HOUSE', 'Subtype of property': 'HOUSE', 'Price': 499000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 155, 'Fully equipped kitchen': 'USA_HYPER_EQUIPPED', 'Furnished': False, 'Open fire': False, 'Terrace': True, 'Terrace area': 27, 'Garden': None, 'Garden area': None, 'Surface of the land': 177, 'Surface area of the plot of land': 177, 'Number of facades ': 2, 'Swimming pool': False, 'State of the building': 'GOOD'}, {'id': 8556707, 'url': 'https://www.immoweb.be/en/classified/8556707', 'Locality': '1140', 'Type of property': 'APARTMENT_GROUP', 'Subtype of property': 'APARTMENT_GROUP', 'Price': None, 'Type of sale': 'group_sale', 'Number of rooms': None, 'Area': None, 'Fully equipped kitchen': None, 'Furnished': None, 'Open fire': False, 'Terrace': None, 'Terrace area': None, 'Garden': None, 'Garden a

# Save information into CSV

In [19]:
save_to_csv(df_element_list,'df_immo.csv')

In [20]:
print("Done")
main_end = time.time()  #Test
print(main_end-main_start)

Done
61.603904724121094
